In [1]:
import keras
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import CSVLogger, ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.utils.vis_utils import plot_model
from keras.applications.vgg19 import decode_predictions
from keras.layers.normalization import BatchNormalization

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import os

import tensorflow
from time import time

import pandas

import pickle as pk

from keras.wrappers.scikit_learn import KerasClassifier

seed = 7

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
np.random.seed(seed)

In [3]:
data_dir = '/data/oxford102/train'
img_width, img_height = 256, 256
batch_size = 32
epochs = 100
nr_categories = 102

In [4]:
img_width, img_height = 256, 256
train_data_dir = "/data/oxford102/train/"
validation_data_dir = "/data/oxford102/train/"
nb_train_samples = 4604
nb_validation_samples = 1094 

In [5]:
input_processor = keras.applications.vgg19.preprocess_input

In [6]:
train_val_datagen = ImageDataGenerator(preprocessing_function=input_processor, 
                                           validation_split=0.2)

train_val_datagen_aug = ImageDataGenerator(
        #rescale=1. / 255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        preprocessing_function=input_processor,
        validation_split=0.2)

In [7]:

    
train_generator = train_val_datagen.flow_from_directory(
        train_data_dir,  # this is the target directory
        target_size=(img_width, img_height),  # all images will be resized to 250x250
        batch_size=batch_size,
        subset="training",
        class_mode='categorical')

validation_generator = train_val_datagen.flow_from_directory(
        train_data_dir,  # this is the target directory
        target_size=(img_width, img_height),  # all images will be resized to 250x250
        subset="validation",
        batch_size=batch_size,
        class_mode='categorical')


Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.


In [8]:
#from keras import regularizers

k.set_learning_phase(0)

network_name = "vgg19"
img_width, img_height = (256, 256)
if network_name == "vgg16":
    base_model = keras.applications.vgg16.VGG16(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
elif network_name == "vgg19":
    base_model = keras.applications.vgg19.VGG19(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
else:
    raise Exception("check your network name")

for layer in base_model.layers[:]:
    layer.trainable = False

    #Adding custom Layers 
k.set_learning_phase(1)
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation="relu", 
          #kernel_regularizer=regularizers.l2(0.01),
         #       activity_regularizer=regularizers.l1(0.001)
         )(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x, training=True)
#x = Dense(102, activation="relu")(x)
predictions = Dense(nr_categories, activation="softmax")(x)

_model = Model(input = base_model.input, output = predictions)

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:30: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [9]:
_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0         
__________

In [10]:
# _model.compile(loss = "categorical_crossentropy", optimizer = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0), metrics=["accuracy"])


In [11]:
#run_training(model_name, _model, train_generator, validation_generator, params, num_train_img, num_val_img):
    

model_name = "vgg19"
num_train_img = 4604
num_val_img = 1094
params = {'network_name': 'vgg19',
         'image_aug': False,
         'optimizer': 'SGD',
          'optimizer_params': None, 
         'batch_size': 512,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}


np.random.seed(seed)
log_time = time()
params['log_time'] = log_time
batch_size = params.get("batch_size")

_model.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.001, momentum=0.9), metrics=["accuracy"])


base = '/data/oxford102/experiments'
path = os.path.join(base, str(log_time))
checkpoint = ModelCheckpoint(os.path.join(path, "{}_{}.h5".format(model_name, log_time)), monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=1, mode='auto')
tensorboard = TensorBoard(log_dir="logs/{}".format(log_time), histogram_freq=0, write_graph=True, write_images=True)
csv_logger = CSVLogger(os.path.join(path, "{}_{}.csv".format(model_name, log_time)), append=True, separator=';')

try:
    if not os.path.exists(path):
        os.makedirs(path)
    history_callback = _model.fit_generator(
            train_generator,
            steps_per_epoch=num_train_img // batch_size,
            epochs=params.get("epochs"),
            validation_data=validation_generator,
            validation_steps=num_val_img // batch_size,
            callbacks = [checkpoint, early, tensorboard, csv_logger])
except Exception as e:
    raise(e)
finally:
    pk.dump(params, open("experimental_params/experiments_{}.pk".format(log_time), "wb"), protocol=pk.HIGHEST_PROTOCOL)
    _model.save_weights(os.path.join(path, 'model_{}_weights_final_{}.h5'.format(model_name, log_time)))  # always save your weights after training or during training
    print(params)
    params

Epoch 1/100
8/8 [==============================] - 10s 1s/step - loss: 5.4749 - acc: 0.0156 - val_loss: 5.1211 - val_acc: 0.0312

Epoch 00001: val_acc improved from -inf to 0.03125, saving model to /data/oxford102/experiments/1543115593.83544/vgg19_1543115593.83544.h5
Epoch 2/100
8/8 [==============================] - 5s 597ms/step - loss: 4.8959 - acc: 0.0273 - val_loss: 4.2551 - val_acc: 0.0938

Epoch 00002: val_acc improved from 0.03125 to 0.09375, saving model to /data/oxford102/experiments/1543115593.83544/vgg19_1543115593.83544.h5
Epoch 3/100
8/8 [==============================] - 5s 600ms/step - loss: 4.1104 - acc: 0.1406 - val_loss: 4.2952 - val_acc: 0.1719

Epoch 00003: val_acc improved from 0.09375 to 0.17188, saving model to /data/oxford102/experiments/1543115593.83544/vgg19_1543115593.83544.h5
Epoch 4/100
8/8 [==============================] - 8s 975ms/step - loss: 3.8163 - acc: 0.1825 - val_loss: 3.9548 - val_acc: 0.2344

Epoch 00004: val_acc improved from 0.17188 to 0.234

In [12]:
model_name = "vgg19"
num_train_img = 4604
num_val_img = 1094

params = {'network_name': 'vgg19',
         'image_aug': False,
         'optimizer': 'SGD',
          'optimizer_params': None, 
         'batch_size': 128,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}


np.random.seed(seed)
log_time = time()
params['log_time'] = log_time
batch_size = params.get("batch_size")

_model.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.001, momentum=0.9), metrics=["accuracy"])


base = '/data/oxford102/experiments'
path = os.path.join(base, str(log_time))
checkpoint = ModelCheckpoint(os.path.join(path, "{}_{}.h5".format(model_name, log_time)), monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=1, mode='auto')
tensorboard = TensorBoard(log_dir="logs/{}".format(log_time), histogram_freq=0, write_graph=True, write_images=True)
csv_logger = CSVLogger(os.path.join(path, "{}_{}.csv".format(model_name, log_time)), append=True, separator=';')

try:
    if not os.path.exists(path):
        os.makedirs(path)
    history_callback = _model.fit_generator(
            train_generator,
            steps_per_epoch=num_train_img // batch_size,
            epochs=params.get("epochs"),
            validation_data=validation_generator,
            validation_steps=num_val_img // batch_size,
            callbacks = [checkpoint, early, tensorboard, csv_logger])
except Exception as e:
    raise(e)
finally:
    pk.dump(params, open("experimental_params/experiments_{}.pk".format(log_time), "wb"), protocol=pk.HIGHEST_PROTOCOL)
    _model.save_weights(os.path.join(path, 'model_{}_weights_final_{}.h5'.format(model_name, log_time)))  # always save your weights after training or during training
    print(params)
    params

Epoch 1/100
35/35 [==============================] - 20s 582ms/step - loss: 1.6284 - acc: 0.6644 - val_loss: 2.1433 - val_acc: 0.4961

Epoch 00001: val_acc improved from -inf to 0.49609, saving model to /data/oxford102/experiments/1543115732.377062/vgg19_1543115732.377062.h5
Epoch 2/100
35/35 [==============================] - 20s 557ms/step - loss: 1.4780 - acc: 0.6848 - val_loss: 2.0189 - val_acc: 0.5312

Epoch 00002: val_acc improved from 0.49609 to 0.53125, saving model to /data/oxford102/experiments/1543115732.377062/vgg19_1543115732.377062.h5
Epoch 3/100
35/35 [==============================] - 20s 560ms/step - loss: 1.3727 - acc: 0.7179 - val_loss: 1.8655 - val_acc: 0.5430

Epoch 00003: val_acc improved from 0.53125 to 0.54297, saving model to /data/oxford102/experiments/1543115732.377062/vgg19_1543115732.377062.h5
Epoch 4/100
35/35 [==============================] - 19s 556ms/step - loss: 1.2774 - acc: 0.7473 - val_loss: 1.6994 - val_acc: 0.6055

Epoch 00004: val_acc improved f

In [13]:
model_name = "vgg19"
num_train_img = 4604
num_val_img = 1094
params = {'network_name': 'vgg19',
         'image_aug': False,
         'optimizer': 'SGD',
          'optimizer_params': None, 
         'batch_size': 64,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}


np.random.seed(seed)
log_time = time()
params['log_time'] = log_time
batch_size = params.get("batch_size")

_model.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.001, momentum=0.9), metrics=["accuracy"])


base = '/data/oxford102/experiments'
path = os.path.join(base, str(log_time))
checkpoint = ModelCheckpoint(os.path.join(path, "{}_{}.h5".format(model_name, log_time)), monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=1, mode='auto')
tensorboard = TensorBoard(log_dir="logs/{}".format(log_time), histogram_freq=0, write_graph=True, write_images=True)
csv_logger = CSVLogger(os.path.join(path, "{}_{}.csv".format(model_name, log_time)), append=True, separator=';')

try:
    if not os.path.exists(path):
        os.makedirs(path)
    history_callback = _model.fit_generator(
            train_generator,
            steps_per_epoch=num_train_img // batch_size,
            epochs=params.get("epochs"),
            validation_data=validation_generator,
            validation_steps=num_val_img // batch_size,
            callbacks = [checkpoint, early, tensorboard, csv_logger])
except Exception as e:
    raise(e)
finally:
    pk.dump(params, open("experimental_params/experiments_{}.pk".format(log_time), "wb"), protocol=pk.HIGHEST_PROTOCOL)
    _model.save_weights(os.path.join(path, 'model_{}_weights_final_{}.h5'.format(model_name, log_time)))  # always save your weights after training or during training
    print(params)
    params

Epoch 1/100
71/71 [==============================] - 40s 570ms/step - loss: 0.1773 - acc: 0.9921 - val_loss: 1.1752 - val_acc: 0.7371

Epoch 00001: val_acc improved from -inf to 0.73713, saving model to /data/oxford102/experiments/1543116077.08352/vgg19_1543116077.08352.h5
Epoch 2/100
71/71 [==============================] - 40s 560ms/step - loss: 0.1789 - acc: 0.9899 - val_loss: 1.1421 - val_acc: 0.7353

Epoch 00002: val_acc did not improve from 0.73713
Epoch 3/100
71/71 [==============================] - 40s 562ms/step - loss: 0.1369 - acc: 0.9974 - val_loss: 1.1117 - val_acc: 0.7408

Epoch 00003: val_acc improved from 0.73713 to 0.74081, saving model to /data/oxford102/experiments/1543116077.08352/vgg19_1543116077.08352.h5
Epoch 4/100
71/71 [==============================] - 40s 559ms/step - loss: 0.1328 - acc: 0.9974 - val_loss: 1.0795 - val_acc: 0.7684

Epoch 00004: val_acc improved from 0.74081 to 0.76838, saving model to /data/oxford102/experiments/1543116077.08352/vgg19_1543116

In [ ]:
model_name = "vgg19"
num_train_img = 4604
num_val_img = 1094
params = {'network_name': 'vgg19',
         'image_aug': False,
         'optimizer': 'SGD',
          'optimizer_params': None, 
         'batch_size': 32,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}


np.random.seed(seed)
log_time = time()
params['log_time'] = log_time
batch_size = params.get("batch_size")

_model.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.001, momentum=0.9), metrics=["accuracy"])


base = '/data/oxford102/experiments'
path = os.path.join(base, str(log_time))
checkpoint = ModelCheckpoint(os.path.join(path, "{}_{}.h5".format(model_name, log_time)), monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=1, mode='auto')
tensorboard = TensorBoard(log_dir="logs/{}".format(log_time), histogram_freq=0, write_graph=True, write_images=True)
csv_logger = CSVLogger(os.path.join(path, "{}_{}.csv".format(model_name, log_time)), append=True, separator=';')

try:
    if not os.path.exists(path):
        os.makedirs(path)
    history_callback = _model.fit_generator(
            train_generator,
            steps_per_epoch=num_train_img // batch_size,
            epochs=params.get("epochs"),
            validation_data=validation_generator,
            validation_steps=num_val_img // batch_size,
            callbacks = [checkpoint, early, tensorboard, csv_logger])
except Exception as e:
    raise(e)
finally:
    pk.dump(params, open("experimental_params/experiments_{}.pk".format(log_time), "wb"), protocol=pk.HIGHEST_PROTOCOL)
    _model.save_weights(os.path.join(path, 'model_{}_weights_final_{}.h5'.format(model_name, log_time)))  # always save your weights after training or during training
    print(params)
    params

# FINETUNE TOP LAYERS

In [14]:
for i, layer in enumerate(base_model.layers[11:]):
    print(i, layer.name)
    layer.trainable = True

0 block3_pool
1 block4_conv1
2 block4_conv2
3 block4_conv3
4 block4_conv4
5 block4_pool
6 block5_conv1
7 block5_conv2
8 block5_conv3
9 block5_conv4
10 block5_pool


In [15]:
model_name = "vgg19"
num_train_img = 4604
num_val_img = 1094
params = {'network_name': 'vgg19',
         'image_aug': False,
         'optimizer': 'SGD',
          'optimizer_params': None, 
         'batch_size': 64,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}


np.random.seed(seed)
log_time = time()
params['log_time'] = log_time
batch_size = params.get("batch_size")

_model.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.001, momentum=0.9), metrics=["accuracy"])


base = '/data/oxford102/experiments'
path = os.path.join(base, str(log_time))
checkpoint = ModelCheckpoint(os.path.join(path, "{}_{}.h5".format(model_name, log_time)), monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=1, mode='auto')
tensorboard = TensorBoard(log_dir="logs/{}".format(log_time), histogram_freq=0, write_graph=True, write_images=True)
csv_logger = CSVLogger(os.path.join(path, "{}_{}.csv".format(model_name, log_time)), append=True, separator=';')

try:
    if not os.path.exists(path):
        os.makedirs(path)
    history_callback = _model.fit_generator(
            train_generator,
            steps_per_epoch=num_train_img // batch_size,
            epochs=params.get("epochs"),
            validation_data=validation_generator,
            validation_steps=num_val_img // batch_size,
            callbacks = [checkpoint, early, tensorboard, csv_logger])
except Exception as e:
    raise(e)
finally:
    pk.dump(params, open("experimental_params/experiments_{}.pk".format(log_time), "wb"), protocol=pk.HIGHEST_PROTOCOL)
    _model.save_weights(os.path.join(path, 'model_{}_weights_final_{}.h5'.format(model_name, log_time)))  # always save your weights after training or during training
    print(params)
    params

Epoch 1/100
71/71 [==============================] - 58s 822ms/step - loss: 0.1295 - acc: 0.9947 - val_loss: 1.0170 - val_acc: 0.7555

Epoch 00001: val_acc improved from -inf to 0.75551, saving model to /data/oxford102/experiments/1543116375.9565437/vgg19_1543116375.9565437.h5
Epoch 2/100
71/71 [==============================] - 55s 778ms/step - loss: 0.1532 - acc: 0.9912 - val_loss: 0.9671 - val_acc: 0.7721

Epoch 00002: val_acc improved from 0.75551 to 0.77206, saving model to /data/oxford102/experiments/1543116375.9565437/vgg19_1543116375.9565437.h5
Epoch 3/100
71/71 [==============================] - 55s 779ms/step - loss: 0.0821 - acc: 0.9982 - val_loss: 0.9087 - val_acc: 0.7739

Epoch 00003: val_acc improved from 0.77206 to 0.77390, saving model to /data/oxford102/experiments/1543116375.9565437/vgg19_1543116375.9565437.h5
Epoch 4/100
71/71 [==============================] - 55s 780ms/step - loss: 0.0792 - acc: 0.9974 - val_loss: 0.8552 - val_acc: 0.8051

Epoch 00004: val_acc impr